In [1]:
# importing libraries

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split
import random


porter = PorterStemmer()
snowball = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Importing all functions from final_project_functions.py
from src.final_project_functions import *

# Now you can use the functions in your code

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\e.sansebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
pip install regex

In [4]:
# getting the data

data = pd.read_csv("../data/data_resampled.csv")
data.head()

,review_title,review_rating,review_content,source,sentiment
0,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,positive
1,awesome experience,5,awesome experience we had to use their custome...,expondo,positive
2,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,positive
3,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,positive
4,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,negative


In [41]:
data["id"] = data.index

In [42]:
def convert_all_to_str(df):
    df_copy = df.copy()  # Create a copy of the DataFrame to avoid the warning
    for column in df_copy.columns:
        df_copy[column] = df_copy[column].astype(str)
    return df_copy


In [43]:
data = convert_all_to_str(data)

In [44]:
data.shape

(15375, 7)

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15375 entries, 0 to 15374
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   review_title      15375 non-null  object
 1   review_rating     15375 non-null  object
 2   review_content    15375 non-null  object
 3   source            15375 non-null  object
 4   sentiment         15375 non-null  object
 5   review_processed  15375 non-null  object
 6   id                15375 non-null  object
dtypes: object(7)
memory usage: 840.9+ KB


In [46]:
data = data[data['review_content'].isna() == False]
data['review_content'].isna().sum()

0

In [47]:
import pandas as pd

def count_whitespace_records(df):
    whitespace_counts = {}

    for column in df.columns:
        # Check for white spaces in the column
        empty_values = df[column].eq('') | df[column].eq(' ') | df[column].eq('  ') | df[column].str.isspace()

        # Count the number of records with white spaces
        whitespace_count = empty_values.sum()

        # Add the column and count to the dictionary
        whitespace_counts[column] = whitespace_count

    return whitespace_counts

In [48]:
# Example usage:
# Assuming 'data' is your DataFrame
result = count_whitespace_records(data)
print(result)

{'review_title': 0, 'review_rating': 0, 'review_content': 0, 'source': 0, 'sentiment': 0, 'review_processed': 0, 'id': 0}


In [49]:
data.dtypes

review_title        object
review_rating       object
review_content      object
source              object
sentiment           object
review_processed    object
id                  object
dtype: object

In [50]:
data["review_content"] = data["review_content"].apply(lambda x: clean_up(x))

In [51]:
from tqdm.notebook import tqdm

tqdm.pandas(desc="Processing", leave=False)
data["review_processed"] = data["review_content"].progress_apply(lambda x: remove_stopwords(tokenize(x)))

Processing:   0%|          | 0/15375 [00:00<?, ?it/s]

In [52]:
data.head()

,review_title,review_rating,review_content,source,sentiment,review_processed,id
0,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,positive,"[ordered, tig, wig, mma, welder, model, wigma,...",0
1,awesome experience,5,awesome experience we had to use their custome...,expondo,positive,"[awesome, experience, use, customer, support, ...",1
2,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,positive,"[product, arrived, time, assembly, details, pe...",2
3,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,positive,"[good, fast, delivery, amazing, standing, desk...",3
4,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,negative,"[bad, invoice, dph, invoice, shipping, slovaki...",4


In [53]:
def stem_words(words_list):
    snow_stemmer = SnowballStemmer(language='english')
    stemmed_list = [snow_stemmer.stem(word) if snow_stemmer.stem(word) in words.words() else word for word in words_list]
    return stemmed_list

tqdm.pandas(desc="Lemmatization", leave=False)

def lemmatize_words(words_list):
    lem = WordNetLemmatizer()
    lemmatized_list = [lem.lemmatize(word) for word in words_list]
    return lemmatized_list

In [54]:
#data["review_processed"] = data["review_processed"].progress_apply(stem_words)

In [55]:
data["review_processed"] = data["review_processed"].progress_apply(lemmatize_words)

Lemmatization:   0%|          | 0/15375 [00:00<?, ?it/s]

In [56]:
data.head()

,review_title,review_rating,review_content,source,sentiment,review_processed,id
0,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,positive,"[ordered, tig, wig, mma, welder, model, wigma,...",0
1,awesome experience,5,awesome experience we had to use their custome...,expondo,positive,"[awesome, experience, use, customer, support, ...",1
2,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,positive,"[product, arrived, time, assembly, detail, per...",2
3,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,positive,"[good, fast, delivery, amazing, standing, desk...",3
4,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,negative,"[bad, invoice, dph, invoice, shipping, slovaki...",4


### Creating Bag of Words

In [57]:
# flattening the processed column in order to get a list of all the words in the column

all_words = [word for word in data['review_processed'] for word in word]
print(all_words[:50])


['ordered', 'tig', 'wig', 'mma', 'welder', 'model', 'wigma', 'pdc', 'shipment', 'austria', 'quick', 'tracking', 'delivery', 'gls', 'perfect', 'welding', 'devicefirst', 'welding', 'test', 'stick', 'electrode', 'mma', 'fine', 'ignition', 'arc', 'stable', 'measured', 'current', 'fluke', 'clamp', 'meter', 'checked', 'deviation', 'ampere', 'measured', 'current', 'almost', 'exactly', 'adjusted', 'one', 'lower', 'current', 'like', 'ampere', 'measured', 'current', 'slightly', 'lower', 'amp', 'think']


In [58]:
fdist = FreqDist(all_words)
fdist

FreqDist({'service': 6324, 'customer': 4890, 'day': 4789, 'delivery': 4683, 'time': 4169, 'order': 3885, 'would': 3608, 'get': 3149, 'item': 3134, 'good': 3078, ...})

In [59]:
# getting the keys from the frequency dictionary (the words) from the top 5000 most frequent words that appear on tweets

word_features = list(fdist.keys())[:5000]

In [60]:
display(word_features[:50])

['ordered',
 'tig',
 'wig',
 'mma',
 'welder',
 'model',
 'wigma',
 'pdc',
 'shipment',
 'austria',
 'quick',
 'tracking',
 'delivery',
 'gls',
 'perfect',
 'welding',
 'devicefirst',
 'test',
 'stick',
 'electrode',
 'fine',
 'ignition',
 'arc',
 'stable',
 'measured',
 'current',
 'fluke',
 'clamp',
 'meter',
 'checked',
 'deviation',
 'ampere',
 'almost',
 'exactly',
 'adjusted',
 'one',
 'lower',
 'like',
 'slightly',
 'amp',
 'think',
 'pretty',
 'good',
 'nonprofessional',
 'device',
 'far',
 'planned',
 'near',
 'future',
 'customer']

In [61]:
def find_features(document, word_features):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [62]:
# applying the function

featuresets = [(find_features(review, word_features), sentiment) for review, sentiment in zip(data['review_processed'], data['sentiment'])]

In [63]:
featuresets[0]

({'ordered': True,
  'tig': True,
  'wig': True,
  'mma': True,
  'welder': True,
  'model': True,
  'wigma': True,
  'pdc': True,
  'shipment': True,
  'austria': True,
  'quick': True,
  'tracking': True,
  'delivery': True,
  'gls': True,
  'perfect': True,
  'welding': True,
  'devicefirst': True,
  'test': True,
  'stick': True,
  'electrode': True,
  'fine': True,
  'ignition': True,
  'arc': True,
  'stable': True,
  'measured': True,
  'current': True,
  'fluke': True,
  'clamp': True,
  'meter': True,
  'checked': True,
  'deviation': True,
  'ampere': True,
  'almost': True,
  'exactly': True,
  'adjusted': True,
  'one': True,
  'lower': True,
  'like': True,
  'slightly': True,
  'amp': True,
  'think': True,
  'pretty': True,
  'good': True,
  'nonprofessional': True,
  'device': True,
  'far': True,
  'planned': True,
  'near': True,
  'future': True,
  'customer': True,
  'supporti': True,
  'question': True,
  'prompt': True,
  'response': True,
  'polite': True,
  'thi

In [64]:
data.tail()

,review_title,review_rating,review_content,source,sentiment,review_processed,id
15370,crappy product not fit for purpose,3,factory tightened bolts couldn t untighten,mowdirect_neutral,neutral,"[factory, tightened, bolt, untighten]",15370
15371,beware,3,great place to buy products if dealing directl...,amazon_neutral,neutral,"[great, place, buy, product, dealing, directly...",15371
15372,product is fine and delivered in a,3,product is fine and delivered in a timely fash...,mowdirect_neutral,neutral,"[product, fine, delivered, timely, fashion, ho...",15372
15373,a very professional supplier,3,i found the crossover to a professional suppli...,trustpilot_mix_data,neutral,"[found, crossover, professional, supplier, int...",15373
15374,cheap prices,3,cheap pricesshipping feesglobal marketing company,amazon_neutral,neutral,"[cheap, pricesshipping, feesglobal, marketing,...",15374


In [73]:
# since when I selected the 10000 rows that were positive and 10000 negative and concatenated them, I should shuffle it otherwise my train/test split
# might only have one type of tweet

random.shuffle(featuresets)

In [74]:
# train/test split

train_set, test_set = train_test_split(featuresets, train_size=0.8, random_state=42)

In [76]:
import pickle
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.tokenize import word_tokenize

# Assuming you have already defined your training data in the 'train_set' variable

# Training the model
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Saving the trained classifier to a pickle file
with open('naive_bayes_classifier.pickle', 'wb') as file:
    pickle.dump(classifier, file)

# Optionally, you can also save the word features used in the training data
# Assuming 'word_features' is the list of features used in the training
with open('word_features.pickle', 'wb') as file:
    pickle.dump(word_features, file)


In [77]:
classifier.show_most_informative_features()

Most Informative Features
                  amazon = True           neutra : positi =    186.4 : 1.0
                   prime = True           neutra : negati =     64.6 : 1.0
                    rude = True           negati : positi =     58.2 : 1.0
               unhelpful = True           negati : positi =     51.5 : 1.0
               beautiful = True           positi : negati =     40.0 : 1.0
               appalling = True           negati : neutra =     38.1 : 1.0
             investigate = True           negati : positi =     37.4 : 1.0
                december = True           negati : positi =     37.4 : 1.0
                  refund = True           negati : positi =     34.4 : 1.0
                   waste = True           negati : positi =     33.0 : 1.0


In [78]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, test_set))*100)

Classifier accuracy percent: 67.77235772357724
